In [1]:
from tqdm import tqdm
import os
from glob import glob
import re
import numpy as np
import pandas as pd

from my_module import *

In [2]:
CSVs = glob('dataframes/*.csv')
CSVs

['dataframes\\pc_games.csv']

In [3]:
platform_df = pd.read_csv('dataframes\\pc_games.csv').drop('Unnamed: 0', axis=1)
platform_df

,Name,Link
0,disco-elysium-the-final-cut,https://www.metacritic.com/game/pc/disco-elysi...
1,half-life-2,https://www.metacritic.com/game/pc/half-life-2
2,grand-theft-auto-v,https://www.metacritic.com/game/pc/grand-theft...
3,the-orange-box,https://www.metacritic.com/game/pc/the-orange-box
4,half-life,https://www.metacritic.com/game/pc/half-life
...,...,...
5286,flatout-3-chaos-destruction,https://www.metacritic.com/game/pc/flatout-3-c...
5287,infestation-survivor-stories-the-war-z,https://www.metacritic.com/game/pc/infestation...
5288,leisure-suit-larry-box-office-bust,https://www.metacritic.com/game/pc/leisure-sui...
5289,alone-in-the-dark-illumination,https://www.metacritic.com/game/pc/alone-in-th...


In [4]:
def try_except(item): # this function is for number extraction from a list containing str(numbers) and str(mix of letters and numbers).
    try:
        int(item)
        return 1
    except: return 0

In [5]:
def dataframe_creator(df):
    global urls, url
    new_cols = ['Distributer', 'Developer', 'Release Date', 'Also On', 'Summary', 
                'Metascore', 'Meta Status', '# of Critic Reviews', 'Critic Positive', 'Critic Mixed', 'Critic Negative', 
                'Userscore', 'Userscore Status', '# of User Reviews', 'User Positive', 'User Mixed', 'User Negative', 
                'Genres', '# Of Players', 'Rating', 'Awards']
    for col in new_cols: df[col] = np.nan # creating empty columns

    df['Status'] = np.nan
    urls = df['Link']
    for counter,url in tqdm(list(enumerate(urls))):

        try: driver = open_url(url)
        except: 
            df['Status'].iloc[counter] = 'WebDriverException'
            continue
        

        """Expand the summary if exist"""
        try: driver.find_element(by=By.CLASS_NAME, value='toggle_expand').click() # click on 'expand' button for 'summary' if exist.
        except: pass

        """Get the scores"""
        try: 
            scores = driver.find_element(by=By.CLASS_NAME, value='product_scores').text.lower().split('\n')
            metascore = scores[1] ; df['Metascore'].iloc[counter] = metascore
            meta_status = scores[2] ; df['Meta Status'].iloc[counter] = meta_status
            critic_reviews_num = re.sub("[^0-9]","",scores[3]) ; df['# of Critic Reviews'].iloc[counter] = critic_reviews_num
            userscore = scores[6] ; df['Userscore'].iloc[counter] = userscore
            userscore_status = scores[7] ; df['Userscore Status'].iloc[counter] = userscore_status
            user_reviews_num = re.sub("[^0-9]","",scores[8]) ; df['# of User Reviews'].iloc[counter] = user_reviews_num
        except: pass

        """Score distribution (2 parts: critics, users)"""
        try: 
            score_dist = driver.find_elements(by=By.CLASS_NAME, value='score_distribution')
            crt_pos, crt_mix, crt_neg = [item for item in score_dist[0].text.replace(',','').split('\n') if try_except(item)==1]
            df['Critic Positive'].iloc[counter] = crt_pos
            df['Critic Mixed'].iloc[counter] = crt_mix
            df['Critic Negative'].iloc[counter] = crt_neg
            usr_pos, usr_mix, usr_neg = [item for item in score_dist[1].text.replace(',','').split('\n') if try_except(item)==1]
            df['User Positive'].iloc[counter] = usr_pos
            df['User Mixed'].iloc[counter] = usr_mix
            df['User Negative'].iloc[counter] = usr_neg
        except: pass

        """Awards"""
        try:
            awards = get_element_by_xpath(driver, '//*[@id="main"]/div/div[3]/div[2]/div[1]/div[2]').text
            if awards[0]=='#': df['Awards'].iloc[counter] = awards
            else: pass
        except: pass
        
        """Summary details (2 parts)"""
        try:
            sum_det = driver.find_elements(by=By.CLASS_NAME, value='summary_details')
            """part 1"""
            sum_det_1 = sum_det[0].text.split('\n')
            distributer = sum_det_1[0] ; df['Distributer'].iloc[counter] = distributer
            release_date = sum_det_1[1].split(': ')[-1] ; df['Release Date'].iloc[counter] = release_date
            try: also_on = sum_det_1[2].split(': ')[-1] ; df['Also On'].iloc[counter] = also_on
            except: pass
            """Part 2"""
            summary = sum_det[1].text[9:] ; df['Summary'].iloc[counter] = summary
        except: pass
        """Other summary details"""
        try: developer = driver.find_element(by=By.CLASS_NAME, value='developer').text[11:] ; df['Developer'].iloc[counter] = developer
        except: pass

        """Genres"""
        try: genres = driver.find_element(by=By.CLASS_NAME, value='product_genre').text[10:] ; df['Genres'].iloc[counter] = genres
        except: pass
        
        """# of players"""
        try: 
            numOfplayers = driver.find_element(by=By.CLASS_NAME, value='product_players').text
            if 'no online' in numOfplayers.lower(): numOfplayers = 'No Online Multiplayer'
            else: numOfplayers = re.sub("[^0-9]","",numOfplayers)
            df['# Of Players'].iloc[counter] = numOfplayers
        except: pass
        
        """Rating"""
        try: rating = driver.find_element(by=By.CLASS_NAME, value='product_rating').text[8:] ; df['Rating'].iloc[counter] = rating
        except: pass

        driver.close()

    return df

In [6]:
df = dataframe_creator(platform_df)
df.to_csv('dataframes/pc_details.csv')

100%|██████████| 5291/5291 [10:05:47<00:00,  6.87s/it]   
